In [58]:
import json
from collections import defaultdict
import pandas as pd

def convert_result_to_df(result):
    data_for_df = []
    total_dimension_prompt_counts = [0, 0]
    for class_name, stats in result.items():
        total_dimension_prompt_counts[0] += stats["dimension_prompt"][0]
        total_dimension_prompt_counts[1] += stats["dimension_prompt"][1]
        data_for_df.append({
            "Class": class_name,
            "Count": int(stats["count"]),
            "Dimension Prompt": stats["dimension_prompt"],
            "Dimensions": stats["dimensions"],
            "Images": stats["img_id"],
            "Parent Dataset": list(set(stats["parent_dataset"])),
            "Annotation": list(set(stats["annotation"]))
        })
    df = pd.DataFrame(data_for_df)
    total_count = df["Count"].sum()
    total_images = df["Images"].sum()
    total_row = {
        "Class": "Total",
        "Count": total_count,
        "Dimension Prompt": total_dimension_prompt_counts,
        "Dimensions": "---",
        "Images": total_images,
        "Parent Dataset": "---",
        "Annotation":"---"
    }
    df = df.append(total_row, ignore_index=True)
    return df

def data_statistic(data_list):
    result = defaultdict(lambda: {
        "count": 0, 
        "dimension_prompt": [0, 0], 
        "dimension": ['',''], 
        "img_id": 0, 
        "parent_dataset": [],
        'annotation': []
    })

    for data in data_list:
        class_name = "_".join(data["data_id"].split("_")[:-1])
        
        result[class_name]["count"] += 1
        
        dimension_prompt = data["dimension_prompt"]
        if dimension_prompt[0]:  
            result[class_name]["dimension_prompt"][0] += 1
        if dimension_prompt[1]:  
            result[class_name]["dimension_prompt"][1] += 1
            
        result[class_name]['dimensions'] = [class_name.split('_')[0],class_name.split('_')[1]]
        
        img_id = data['img_id']
        if img_id:
            result[class_name]['img_id'] += 1
        result[class_name]["parent_dataset"].append(data["parent_dataset"][0])
        result[class_name]["annotation"].append(data["parent_dataset"][1])

    # 返回统计结果
    return result

def data_integrity_verification(data):
    return 

In [59]:
subsets = ['t2i','subject_gen']
with open(f'F:\Projects\MOGAI\data\prompts\mogai_t2i_v32_v2.json','r') as f:
    data_list = json.load(f)
df = convert_result_to_df(data_statistic(data_list))
df

C:\Users\Zz\AppData\Local\Temp\ipykernel_11952\1145516941.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(total_row, ignore_index=True)


,Class,Count,Dimension Prompt,Dimensions,Images,Parent Dataset,Annotation
0,IQ-R_IQ-A,500,"[0, 0]","[IQ-R, IQ-A]",500,"[DOCCI, Flickr]",[Origin]
1,IQ-R_TA-C,500,"[0, 0]","[IQ-R, TA-C]",500,"[DOCCI, Flickr]",[Origin]
2,IQ-R_IQ-O,300,"[0, 300]","[IQ-R, IQ-O]",0,[New],[Human]
3,IQ-R_TA-R,500,"[0, 500]","[IQ-R, TA-R]",170,"[New, DOCCI]","[Auto, Origin]"
4,IQ-R_TA-S,500,"[0, 500]","[IQ-R, TA-S]",0,"[COCO, Flickr]",[Human]
...,...,...,...,...,...,...,...
57,TA-S_R-T,300,"[300, 300]","[TA-S, R-T]",0,"[<IQ-R, R-T>]",[Human]
58,TA-S_R-B,200,"[200, 0]","[TA-S, R-B]",0,"[<IQ-R, R-B>]",[Human]
59,D-K_R-T,300,"[0, 300]","[D-K, R-T]",0,"[<IQ-R, R-T>]",[Origin]
60,D-K_R-B,200,"[200, 0]","[D-K, R-B]",0,"[<IQ-R, R-B>]",[Auto]
